# Testing modified function in ZMCIntegral

Start, as usual, with import statements.

In [1]:
import math
import numpy as np
from numba import cuda
# from ZMCintegral import ZMCintegral
import ZMCIntegral
import time
from typing import List
from numba import njit

Now define the constants needed in the function.

In [2]:
mu = 0.1  # Fermi-level
hOmg = 0.5  # Photon energy eV
a = 4  # AA
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
rati = 0.3  # ratio
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))
# print(eE0)
Gamm = 0.005  # Gamma in eV.
KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

In [30]:
@cuda.jit(device=True)
def my_Besselv(v, z):
    # WILL NOT WORK IF v IS NOT AN INTEGER
    # Conditional to handle case of negative v.
    if(v < 0):
        v = abs(v)
        resultsign = (-1) ** v
    else:
        resultsign = 1
    result = 0    
    # Loop to construct Bessel series sum.
    for n in range(0,20):
        sign = (-1)**n
        exp = 2 * n + v
        term = z ** exp
        r = n + v + 1
        if(r == 0):
            r = 1e-15
        denom = math.gamma(r)
        denom = denom * math.factorial(n)
        denom = denom * (2 ** exp)
        term = term / denom * sign
        # print('for ', n, ': ',term)
        result = result + term
        
    return result * resultsign
        
    
@cuda.jit(device=True)
def myHeaviside(z):
    # Wrote this Heaviside expression with it cast in cuda to avoid error below.
    if z <= 0 :
	    return 0
    else :
	    return 1
    
@cuda.jit(device=True)
def Ds_helper1(N, ek, ekq):
    j = 0
    i = -(N - 1) / 2
    
    taninv1kp = np.array(N, dtype='complex')
    taninv1kqp = np.array(N, dtype='complex')
    taninv1km = np.array(N, dtype='complex')
    taninv1kqm = np.array(N, dtype='complex')

    lg1kp = np.array(N, dtype='complex')
    lg1kqp = np.array(N, dtype='complex')
    lg1km = np.array(N, dtype='complex')
    lg1kqm = np.array(N, dtype='complex')

    ferp = np.array(N, dtype='complex')
    ferm = np.array(N, dtype='complex')
    
    while(i < ((N - 1) / 2 + 1)):
        taninv1kp[j] = 2 * math.atan2(Gamm, ek - hOmg / 2 + hOmg * i)
        taninv1kqp[j] = 2 * math.atan2(Gamm, ekq - hOmg / 2 + hOmg * i)
        taninv1km[j] = 2 * math.atan2(Gamm, ek + hOmg / 2 + hOmg * i)
        taninv1kqm[j] = 2 * math.atan2(Gamm, ekq + hOmg / 2 + hOmg * i)

        lg1kp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * i) ** 2)
        lg1kqp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * i) ** 2)
        lg1km[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * i) ** 2)
        lg1kqm[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * i) ** 2)

        ferp[j] = myHeaviside(mu - hOmg / 2 - hOmg * i)
        ferm[j] = myHeaviside(mu + hOmg / 2 - hOmg * i)
        j = j + 1
        i = i + 1
    Ds_list1 = [taninv1kp,taninv1kqp,taninv1km,taninv1kqm,lg1kp,lg1kqp,lg1km,lg1kqm,ferp,ferm]
    return Ds_list1
        
        
@cuda.jit(device=True) 
def modDsN2(x):
    N = 2
    dds = 0
    ds = 0
    ek = A * (math.sqrt((x[0]) ** 2 + (x[1]) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((x[0]) ** 2 + (x[1]) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2) / hOmg ** 2
    
    Ds_list1 = Ds_helper1(N, ek, ekq)
    size_dbl = 2 * N - 1
    
    taninv2k = np.empty(size_dbl)
    taninv2kq = np.empty(size_dbl)

    lg2k = np.empty(size_dbl, dtype='complex')
    lg2kq = np.empty(size_dbl, dtype='complex')

    besk = np.empty(size_dbl)
    beskq = np.empty(size_dbl)

    fac1 = np.empty(size_dbl)
    fac2 = np.empty(size_dbl, dtype='complex')
    fac3 = np.empty(size_dbl, dtype='complex')

    j = 0
    for i in range(-(N - 1), N, 1):
        taninv2k[j] = 2 * math.atan2(Gamm, ek - mu + hOmg * i)
        taninv2kq[j] = 2 * math.atan2(Gamm, ekq - mu + hOmg * i)

        lg2k[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - mu + hOmg * i) ** 2)
        lg2kq[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - mu + hOmg * i) ** 2)

        besk[j] = my_Besselv(i, xk)
        beskq[j] = my_Besselv(i, xkq)

        fac1[j] = ek - ekq + hOmg * i
        fac2[j] = fac1[j] + 2 * complex(0, 1) * Gamm
        fac3[j] = fac2[j] - ek + ekq
        j = j + 1
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    Ds_list1[0][alpha] - taninv2k[s + alpha] - Ds_list1[4][alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    Ds_list1[0][beta] - taninv2k[s + beta] + Ds_list1[4][beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -Ds_list1[1][gamma] + taninv2kq[s + gamma] - Ds_list1[5][gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    Ds_list1[2][alpha] - taninv2k[s + alpha] - Ds_list1[6][alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    Ds_list1[2][beta] - taninv2k[s + beta] + Ds_list1[6][beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -Ds_list1[3][gamma] + taninv2kq[s + gamma] - Ds_list1[7][gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]

                            omint1p = Ds_list1[8][s] * ((p1p + p2p + p3p) / d1)

                            omint1m = Ds_list1[9][s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -Ds_list1[1][gamma] + taninv2kq[s + gamma] - Ds_list1[5][gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -Ds_list1[1][beta] + taninv2kq[s + beta] + Ds_list1[5][beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    Ds_list1[0][alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -Ds_list1[3][gamma] + taninv2kq[s + gamma] - Ds_list1[7][gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -Ds_list1[3][beta] + taninv2kq[s + beta] + Ds_list1[7][beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    Ds_list1[2][alpha] - taninv2k[s + alpha] - Ds_list1[6][alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = Ds_list1[8][s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = Ds_list1[9][s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)

    return dds.real

In [31]:
MC = ZMCIntegral.MCintegral(modDsN2,[[-.785,.785],[-.785,.785],[0,.785]])

MC.depth = 2
MC.sigma_multiplication = 10**10
MC.num_trials = 5
MC.available_GPU=[0]

In [32]:
# obtaining the result
t0 = time.time()
result = MC.evaluate()
t1 = time.time()

# print the formatted result
print('result = %s    std = %s' % (result[0], result[1]))
print('This was calculated in', t1-t0, 's')

Process Process-20:
Traceback (most recent call last):
  File "/home/jalenc/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jalenc/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jalenc/github/my-phys-util/ZMCIntegral/ZMCIntegral.py", line 115, in multi_processing
    result.append(self.MCkernel(domain, i_batch))
  File "/home/jalenc/github/my-phys-util/ZMCIntegral/ZMCIntegral.py", line 377, in MCkernel
    integration_kernel[blockspergrid, threadsperblock](num_loops,                                                           MCresult,                                                           chunk_size,                                                           n_chunk_x,                                                           domain,                                                           domain_range,                                            

FileNotFoundError: [Errno 2] No such file or directory: '/home/jalenc/github/my-phys-util/ZMCIntegral/multi_temp/result0.npy'